In [1]:
api_url = "https://earth-search.aws.element84.com/v1"
from pystac_client import Client
client = Client.open(api_url)
collection = "sentinel-2-l2a"  # Sentinel-2, Level 2A, Cloud Optimized GeoTiffs (COGs)

In [2]:
from pygeotile.tile import Tile
from pygeotile.point import Point as pyPoint
from shapely.geometry import Point, Polygon

tms_x, tms_y, zoom = 4071, 2751, 13
tile = Tile.from_google(google_x=tms_x, google_y=tms_y, zoom=zoom)  # Tile Map Service (TMS) X Y and zoom
print(tile.bounds[0])
polygon = Polygon([
    Point(tile.bounds[0].longitude,tile.bounds[0].latitude),
    Point(tile.bounds[1].longitude,tile.bounds[0].latitude),
    Point(tile.bounds[1].longitude,tile.bounds[1].latitude),
    Point(tile.bounds[0].longitude,tile.bounds[1].latitude)
])

print(tile.tms[0])
print(tile.bounds)
print('Bounds: ',polygon.wkt)

Point(latitude=50.73645513701064, longitude=-1.0986328125000033)
4071
(Point(latitude=50.73645513701064, longitude=-1.0986328125000033), Point(latitude=50.76425935711649, longitude=-1.0546875000000047))
Bounds:  POLYGON ((-1.0986328125000033 50.73645513701064, -1.0546875000000047 50.73645513701064, -1.0546875000000047 50.76425935711649, -1.0986328125000033 50.76425935711649, -1.0986328125000033 50.73645513701064))


In [5]:
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pathlib import Path
import rasterio.plot
import rasterio.mask
import os
import urllib
search = client.search(
    collections=[collection],
    intersects=polygon,
    max_items=100,
    sortby=[
            {"direction": "desc", "field": "properties.datetime"},
            {"direction": "asc", "field": "id"}
    ]
)

dst_crs = 'EPSG:3857'

print(search.matched())
item = [item for item in search.items() if item.properties['eo:cloud_cover'] < 20][0]
# print(item.__dict__)
print(item.properties['eo:cloud_cover'])
print(item.id)
print(item.assets.keys())
print(item)
file_loc = item.id+".tif"
file_dest = str(tms_x)+str(tms_y)+str(zoom)+".tif"
print(item.assets['visual'])
if (not os.path.isfile(file_loc)):
    urllib.request.urlretrieve(item.assets['visual'].href, file_loc)#Would be worth returning Scene Classification Layer (SCL)
with rasterio.open(file_loc) as dataset:
    #rasterio.plot.show(dataset)
    #reprojectn('EPSG:'+str(item.properties['proj:epsg']),'EPSG:3857')
    transform, width, height = calculate_default_transform(
    dataset.crs, dst_crs, dataset.width, dataset.height, *dataset.bounds)
    kwargs = dataset.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    with rasterio.open(file_dest, 'w', **kwargs) as dst:
        for i in range(1, dataset.count + 1):
            reproject(
                source=rasterio.band(dataset, i),
                destination=rasterio.band(dst, i),
                src_transform=dataset.transform,
                src_crs=dataset.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.bilinear)
    for x in range(4071,4073):
        for y in range(2750,2753):
            tile = Tile.from_google(google_x=x, google_y=y, zoom=zoom)  # Tile Map Service (TMS) X Y and zoom
           
            with rasterio.open(file_dest) as src:
                polygon_3857 = Polygon([
                    pyPoint.from_latitude_longitude(latitude=tile.bounds[0].latitude,longitude=tile.bounds[0].longitude).meters,
                    pyPoint.from_latitude_longitude(latitude=tile.bounds[0].latitude,longitude=tile.bounds[1].longitude).meters,
                    pyPoint.from_latitude_longitude(latitude=tile.bounds[1].latitude,longitude=tile.bounds[1].longitude).meters,
                    pyPoint.from_latitude_longitude(latitude=tile.bounds[1].latitude,longitude=tile.bounds[0].longitude).meters,
                ])
                print(polygon_3857)
                print(src.bounds)
                print(src.crs)
                
                Path(f"{str(zoom)}/{str(x)}/{str(y)}").mkdir(parents=True, exist_ok=True)
                file_dest_cropped = f"{str(zoom)}/{str(x)}/{str(y)}/visual.tif"
                out_image, out_transform = rasterio.mask.mask(src, [polygon_3857], crop=True)
                out_meta = src.meta
                out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
                with rasterio.open(file_dest_cropped, "w", **out_meta) as dest:
                    dest.write(out_image)

524
0.009194
S2B_30UXB_20230526_0_L2A
dict_keys(['aot', 'blue', 'coastal', 'granule_metadata', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1', 'rededge2', 'rededge3', 'scl', 'swir16', 'swir22', 'thumbnail', 'tileinfo_metadata', 'visual', 'wvp', 'aot-jp2', 'blue-jp2', 'coastal-jp2', 'green-jp2', 'nir-jp2', 'nir08-jp2', 'nir09-jp2', 'red-jp2', 'rededge1-jp2', 'rededge2-jp2', 'rededge3-jp2', 'scl-jp2', 'swir16-jp2', 'swir22-jp2', 'visual-jp2', 'wvp-jp2'])
<Item id=S2B_30UXB_20230526_0_L2A>
<Asset href=https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/30/U/XB/2023/5/S2B_30UXB_20230526_0_L2A/TCI.tif>
POLYGON ((-122299.24525628237 6579699.394787974, -117407.27544603124 6579699.394787974, -117407.27544603124 6584591.364598225, -122299.24525628237 6584591.364598225, -122299.24525628237 6579699.394787974))
BoundingBox(left=-177141.02370308578, bottom=6520436.993711313, right=1894.787500990933, top=6699916.905084926)
EPSG:3857
POLYGON ((-122299.24525628237 6574807.424977